In [0]:
import warnings
warnings.filterwarnings("ignore")

import csv, math, pickle, os, pdb, gc, random
import pandas as pd
import datetime 
import time 
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
from matplotlib import rcParams
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import warnings
import networkx as nx
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
from tqdm import tqdm

# 1. Reading Data

In [3]:
d = '/content/drive/My Drive/Colab Notebooks/Datasets/Case_Study/Facebook friend recommendation/'
if os.path.isfile(d+'after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist(d+'after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


# 2. Function to Featurize the data

## 2.1 Simiarity Measures

### 2.1.1 Jaccard Index/Similarity

To know in detalis: [link](http://www.statisticshowto.com/jaccard-index/)
\begin{equation}
J(A,B) = \frac{|A\cap B|}{|A \cup B|} 
\end{equation}

In [0]:
#for followees
def jaccard_for_followees(a,b):
    try:
        A = set(train_graph.successors(a))
        B = set(train_graph.successors(b))
        
        if (len(A) == 0)  | (len(B) == 0):
            return 0
    
        A_intersection_B = A.intersection(B)
        A_union_B        = A.union(B)    
        jaccard = len(A_intersection_B)/len(A_union_B)
        
        return jaccard
    except:
        return 0
 

In [5]:
#one test case
print(jaccard_for_followees(273084,1505602))
#node 1635354 not in graph
print(jaccard_for_followees(1635354,1505602))

0.0
0


In [0]:
#for followers
def jaccard_for_followers(a,b):
    try:
        A = set(train_graph.predecessors(a))
        B = set(train_graph.predecessors(b))
        
        if (len(A) == 0)  | (len(B) == 0):
            return 0
    
        A_intersection_B = A.intersection(B)
        A_union_B        = A.union(B)    
        jaccard = len(A_intersection_B)/len(A_union_B)
        
        return jaccard
    except:
        return 0
 

In [7]:
#one test case
print(jaccard_for_followers(273084,1505602))
#node 1635354 not in graph
print(jaccard_for_followers(1635354,1505602))

0.0
0


### 2.1.2 Cosine Similarity (Modified)

* __Otsuka-Ochiai coefficient__
  * To know in deatils click [here](https://en.wikipedia.org/wiki/Cosine_similarity#Otsuka-Ochiai_coefficient).
\begin{equation}
O(A,B) = \frac{|A\cap B|}{\sqrt{|A|\cdot|B|}} 
\end{equation}

In [0]:
#for followees
def Otsuka_Ochiai_coefficient_followees(a,b):
    try:
        A = set(train_graph.successors(a))
        B = set(train_graph.successors(b))
        
        if (len(A) == 0)  | (len(B) == 0):
            return 0
    
        A_intersection_B = A.intersection(B)  
        Otsuka_Ochiai_coef = len(A_intersection_B)/math.sqrt(len(A)*len(B))
        
        return Otsuka_Ochiai_coef
    except:
        return 0

In [9]:
#one test case
print(Otsuka_Ochiai_coefficient_followees(273084,1505602))
#node 1635354 not in graph
print(Otsuka_Ochiai_coefficient_followees(1635354,1505602))

0.0
0


In [0]:
#for followers
def Otsuka_Ochiai_coefficient_followers(a,b):
    try:
        A = set(train_graph.predecessors(a))
        B = set(train_graph.predecessors(b))
        
        if (len(A) == 0)  | (len(B) == 0):
            return 0
    
        A_intersection_B = A.intersection(B)  
        Otsuka_Ochiai_coef = len(A_intersection_B)/math.sqrt(len(A)*len(B))
        
        return Otsuka_Ochiai_coef
    except:
        return 0

In [11]:
#one test case
print(Otsuka_Ochiai_coefficient_followers(273084,1505602))
#node 1635354 not in graph
print(Otsuka_Ochiai_coefficient_followers(1635354,1505602))

0.0
0


## 2.2 Page Ranking

__PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.__
* This is implemented in *[networkx](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)*

Image Source: https://upload.wikimedia.org/wikipedia/en/8/8b/PageRanks-Example.jpg
<img src="https://upload.wikimedia.org/wikipedia/en/8/8b/PageRanks-Example.jpg" width="480">

Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. (The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.




To know in deails: [cilck here](https://en.wikipedia.org/wiki/PageRank)

In [0]:
if not os.path.isfile(d+'fea_sample/page_rank.p'):
    page_rank = nx.pagerank(train_graph, alpha=0.85)
    with open(d+'fea_sample/page_rank.p','wb') as f:
        pickle.dump(page_rank,f)
else:
    with open(d+'fea_sample/page_rank.p','rb') as f:
      page_rank = pickle.load(f)

In [13]:
print('min',page_rank[min(page_rank, key=page_rank.get)])
print('max',page_rank[max(page_rank, key=page_rank.get)])
print('mean',float(sum(page_rank.values())) / len(page_rank))

min 1.6556497245737814e-07
max 2.7098251341935827e-05
mean 5.615699699389075e-07


In [14]:
#for imputing to nodes which are not there in Train data
mean_page_rank = float(sum(page_rank.values())) / len(page_rank)
print(mean_page_rank)

5.615699699389075e-07


## 2.3 Some Other Graph Features

### 2.3.1 Shortest Path

* Getting Shortest path between two nodes.
* If nodes have direct path i.e directly connected then we are removing that edge and calculating path.

**Note:** This will remove the edge between two nodes if there exist a direct edge. So after calling this fucntion you must load your graph again.

In [0]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):

    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [16]:
#testing
print('For existing nodes                 - source = {}, target = {}: {}'.format(77697, 826021, compute_shortest_path_length(77697, 826021)))
print('For node which is not in train data- source = {}, target = {}: {}'.format(669354, 1635354, compute_shortest_path_length(669354,1635354)))

For existing nodes                 - source = 77697, target = 826021: 10
For node which is not in train data- source = 669354, target = 1635354: -1


### 2.3.2 Checking for same community

* Boolean feature: 0 or 1
* Does given a pair of node exist in weakly connected graph or not?

In [0]:
#getting weekly connected edges from graph 
wcc = list(nx.weakly_connected_components(train_graph))

def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(a,b):
        for i in wcc:
            if a in i:
                index= i
                break
        if (b in index):
            if compute_shortest_path_length(a,b)==-1:
                return 0
            else:
                train_graph.add_edge(a,b)
                return 1
        else:
            return 0
    else:
        for i in wcc:
            if a in i:
                index= i
                break
                
        if(b in index):
            return 1
        else:
            return 0

In [18]:
print(belongs_to_same_wcc(861, 1659750))
print(belongs_to_same_wcc(669354,1635354))

0
0


### 2.3.3 Adamic/Adar Index:

Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
\begin{equation}
A(a,b) = \sum_{u \in N(b) \cap N(b)}\frac{1}{log(|N(u)|)}
\end{equation}

**Note:** Modified Adar index is implemented here becasue it is defined for undirected graphs. But we're using this concpet *u ∈ U*, where *U* will be intersection of successors of *x* and *y*. *N(u)* will be set of predessors of *u*. 

In [0]:
#adar index
def calc_adar_in(a,b):
    my_sum = 0
    try:
        A = set(train_graph.successors(a)).union(set(train_graph.predecessors(a)))
        B = set(train_graph.successors(b)).union(set(train_graph.predecessors(b)))
        u = A.intersection(B)
        if len(u) != 0:
            for i in u:
                N_u = set(train_graph.successors(i)).union(train_graph.predecessors(i))
                my_sum += 1/np.log10(len(N_u))
            return my_sum
        else:
            return 0
    except:
        return 0

In [20]:
print(calc_adar_in(1,189226))
print(calc_adar_in(669354,1635354))

0.6705281516444292
0


### 2.3.4 Follows back

In [0]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [22]:
print(follows_back(1,189226))
print(follows_back(669354,1635354))

1
0


### 2.3.5 Katz Centrality

* https://en.wikipedia.org/wiki/Katz_centrality

* https://www.geeksforgeeks.org/katz-centrality-centrality-measure/
 
Katz centrality computes the centrality for a node 
    based on the centrality of its neighbors. It is a 
    generalization of the eigenvector centrality. The
    Katz centrality for node `i` is
 
$$x_i = \alpha \sum_{j} A_{ij} x_j + \beta,$$
where `A` is the adjacency matrix of the graph G 
with eigenvalues $$\lambda$$.

The parameter $$\beta$$ controls the initial centrality and 

$$\alpha < \frac{1}{\lambda_{max}}.$$

In [0]:
if not os.path.isfile(d+'fea_sample/katz.p'):
    katz = nx.katz.katz_centrality(train_graph, alpha=0.005, beta=1)
    with open(d+'fea_sample/katz.p','wb') as f:
        pickle.dump(katz, f)
else:
    with open(d+'fea_sample/katz.p','rb') as f:
        katz = pickle.load(f)

In [24]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007313532503674038
max 0.003394554990800175
mean 0.0007483800934937644


In [25]:
mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

0.0007483800934937644


### 2.3.6 Hits Score

* The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.
* For detail information refer this [link](https://en.wikipedia.org/wiki/HITS_algorithm)

In [0]:
if not os.path.isfile(d+'fea_sample/hits.p'):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    with open(d+'fea_sample/hits.p','wb') as f:
        pickle.dump(hits, f)
else:
    with open(d+'fea_sample/hits.p','rb') as f:
        hits = pickle.load(f)

In [27]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


### 2.3.7  Number of follower and followee feature

In [0]:
def compute_number_features(row):
    a = [0]*7
    try:
        s1 = set(train_graph.predecessors(row['source_node']))
        s2 = set(train_graph.successors(row['source_node']))
    except:
        s1 = set()
        s2 = set()
    
    try:
        d1=set(train_graph.predecessors(row['destination_node']))
        d2=set(train_graph.successors(row['destination_node']))
    except:
        d1 = set()
        d2 = set()
    source      = s1.union(s2)
    destination = d1.union(d2)
    
    a[0], a[1], a[2], a[3] = len(s1), len(s2), len(d1), len(d2)
    a[4], a[5]= len(s1.intersection(d1)), len(s2.intersection(d2))
    a[6] = len(source)*len(destination)
    return a

### 2.3.8 For weight Feature

* Weight Features
 * weight of incoming edges
 * weight of outgoing edges
 * weight of incoming edges + weight of outgoing edges
 * weight of incoming edges * weight of outgoing edges
 * 2*weight of incoming edges + weight of outgoing edges
 * weight of incoming edges + 2*weight of outgoing edges
 
In order to determine the similarity of nodes, an edge weight value was calculated between nodes. Edge weight decreases as the neighbor count goes up. Intuitively, consider one million people following a celebrity on a social network then chances are most of them never met each other or the celebrity. On the other hand, if a user has 30 contacts in his/her social network, the chances are higher that many of them know each other. 

`credit` - Graph-based Features for Supervised Link Prediction
William Cukierski, Benjamin Hamner, Bo Yang

In [29]:
#weight for source and destination of each link
Weight_in = {}
Weight_out = {}
for i in  tqdm(train_graph.nodes()):
    s1=set(train_graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(train_graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|██████████| 1780722/1780722 [00:13<00:00, 129682.46it/s]


### 2.3.9 For SVD Features

In [0]:
#for svd features to get feature vector creating a dict node val and inedx in svd vector
sadj_col = sorted(train_graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

In [31]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1780722, 1780722)
U Shape (1780722, 6)
V Shape (6, 1780722)
s Shape (6,)


### 2.3.10 SVD dot

In [0]:
def svd_dot(row):
    u_s = row[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']].values
    u_d = row[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']].values
    
    v_s = row[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]].values
    v_d = row[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']].values
    
    aa = np.dot(u_s, u_d)
    bb = np.dot(v_s, v_d)
    cc = np.dot(u_s, v_s)
    dd = np.dot(u_d, v_d)
    
    return [aa, bb, cc, dd]

# 3. Featurization

## 3.1 Adding a set of features

* __we will create these each of these features for both train and test data points__

  * **Jaccard Index** 
   * jaccard_followers
   * jaccard_followees
 * **cosine similarity/Otsuka-Ochiai coefficient**  
    * cosine_followers
    * cosine_followees
 * **number of followers/followees** 
    * num_followers_s : of source
    * num_followers_d: of destination
    * num_followees_s: of source
    * num_followees_d: : of destination
    * inter_followers      : number of common followers of source and destination
    * inter_followees   : number of common followees of source and destination
    * preferntial_attachment: |N(a)|*|N(b)|
 * **Adar Index**
   * adar_index
 * **Follows back**
    * follows_back
 * **belongs to same weakly connect components**
    * same_comp
 * **shortest path between source and destination**
    * shortest_path
 * __Weight Features__
   * weight of incoming edges
   * weight of outgoing edges
   * weight of incoming edges + weight of outgoing edges
   * weight of incoming edges * weight of outgoing edges
   * 2*weight of incoming edges + weight of outgoing edges
   * weight of incoming edges + 2*weight of outgoing edges

 * **Page Ranking**
    * page_rank_s: of source
    * page_rank_d: of destination
 * **katz** 
    * katz_s: of source
    * katz_s: of destination
 * **HITS**
    * hubs_s: of source
    * hubs_d: of destination
    * authorities_s: of source
    * authorities_d: of destination
 * **SVD features for both source and destination**
\begin{equation}
A = USV^T
\end{equation}
    * Taking top 6 rows of U and top 6 columns of corresponding source and destnation
    * That makes 6*4 = 24 (total features)
 * **SVD dot** 
    * svd_u_s_d: svd_u_s * svd_u_d
    * svd_v_s_d: svd_v_s * svd_v_d
    * 'svd_u_v_s: svd_u_s * svd_v_s
    * svd_u_v_d: svd_u_d * svd_v_d

**Below fucntion will do all featurization**

In [0]:
def featurize_my_df(df):
    """This will featurize the given dataframe"""
    
    print('\nDoing simiarity featurization....')
    df['jaccard_followers'] = df.apply(lambda row: jaccard_for_followers(row['source_node'],row['destination_node']),axis=1)
    df['jaccard_followees'] = df.apply(lambda row: jaccard_for_followees(row['source_node'],row['destination_node']),axis=1)
    df['cosine_followers']  = df.apply(lambda row: Otsuka_Ochiai_coefficient_followers(row['source_node'],row['destination_node']),axis=1)
    df['cosine_followees']  = df.apply(lambda row: Otsuka_Ochiai_coefficient_followees(row['source_node'],row['destination_node']),axis=1)

    print('\nDoing number of followers and followee based featurization......')
    nums_features = df.apply(compute_number_features, axis = 1)
    cols = ['num_followers_s', 'num_followers_d', 'num_followees_s', 'num_followees_d', 
            'inter_followers', 'inter_followees', 'preferntial_attachment']
    for i, col in enumerate(cols):
        df[col] = list(map(lambda x: x[i], nums_features))

    print('\nDoing adar index, follow back, same community, and shortest path featurization....')
    df['adar_index']    = df.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)
    df['follows_back']  = df.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)
    df['same_comp']     = df.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)
    df['shortest_path'] = df.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)

    print('\nDoing weight featurization....')
    df['weight_in']  = df['destination_node'].apply(lambda x: Weight_in.get(x,mean_weight_in))
    df['weight_out'] = df['source_node'].apply(lambda x: Weight_out.get(x,mean_weight_out))

    df['weight_f1'] = df['weight_in'] + df['weight_out']
    df['weight_f2'] = df['weight_in'] * df['weight_out']
    df['weight_f3'] = 2*df['weight_in'] + 1*df['weight_out']
    df['weight_f4'] = 1*df['weight_in'] + 2*df['weight_out']

    print('\nDoing PageRank, Kartz, and HITS featurization...')
    df['page_rank_s']   = df['source_node'].apply(lambda x:page_rank.get(x,mean_page_rank))
    df['page_rank_d']   = df['destination_node'].apply(lambda x:page_rank.get(x,mean_page_rank))
    df['katz_s']        = df['source_node'].apply(lambda x: katz.get(x,mean_katz))
    df['katz_d']        = df['destination_node'].apply(lambda x: katz.get(x,mean_katz))
    df['hubs_s']        = df['source_node'].apply(lambda x: hits[0].get(x,0))
    df['hubs_d']        = df['destination_node'].apply(lambda x: hits[0].get(x,0))
    df['authorities_s'] = df['source_node'].apply(lambda x: hits[1].get(x,0))
    df['authorities_s'] = df['destination_node'].apply(lambda x: hits[1].get(x,0))

    print('\nDoing SVD featurization.......\n')
    df[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = df['source_node'].apply(lambda x: svd(x, U)).apply(pd.Series)
    df[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = df['destination_node'].apply(lambda x: svd(x, U)).apply(pd.Series)
    df[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6']]= df['source_node'].apply(lambda x: svd(x, V.T)).apply(pd.Series)
    df[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = df['destination_node'].apply(lambda x: svd(x, V.T)).apply(pd.Series)
    
    print('\nDoing SVD dot featurization.....')
    df[['svd_u_s_d', 'svd_v_s_d','svd_u_v_s', 'svd_u_v_d']] = df.apply(svd_dot, axis = 1).apply(pd.Series)

    return df


In [34]:
if os.path.isfile(d+'final_train.csv'):
    df_train = pd.read_csv(d+'final_train.csv')
    print('Shape of featurized train data is: ', df_train.shape)
else:
    # readining randomly only 1m rows from train data: #https://stackoverflow.com/a/22259008/4084039
    n_train =  15100030
    s = 1000000 #desired sample size i.e. 1 million
    skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))
    
    print("Number of rows in the train data file:", n_train)
    print("Number of rows we are going to elimiate in train data are:",len(skip_train))
    
    df_train = pd.read_csv(d+'after_eda/train_after_eda.csv', skiprows=skip_train,  names=['source_node', 'destination_node'])
    df_train['indicator_link'] = pd.read_csv(d+'after_eda/train_y.csv', skiprows=skip_train, names=['indicator_link'])
    print("\nShape of data before featurization ",df_train.shape)

    print('Featurizing train data')
    df_train = featurize_my_df(df_train)
    df_train.to_csv(d+'final_train.csv', index = False)
    print('Shape of train data after featurization is: ', df_train.shape)
df_train.head()

Shape of featurized train data is:  (1000001, 59)


,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,preferntial_attachment,adar_index,follows_back,same_comp,shortest_path,weight_in,weight_out,weight_f1,weight_f2,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,svd_u_s_1,svd_u_s_2,svd_u_s_3,svd_u_s_4,svd_u_s_5,svd_u_s_6,svd_u_d_1,svd_u_d_2,svd_u_d_3,svd_u_d_4,svd_u_d_5,svd_u_d_6,svd_v_s_1,svd_v_s_2,svd_v_s_3,svd_v_s_4,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,svd_u_s_d,svd_v_s_d,svd_u_v_s,svd_u_v_d
0,273084,1505602,1,0.0,0.000000,0.0,0.000000,11,15,6,8,0,0,144,0.000000,0,1,4,0.377964,0.250000,0.627964,0.094491,1.005929,0.877964,2.045290e-06,3.459963e-07,0.000773,0.000756,1.943132e-13,1.941103e-13,2.231877e-15,-1.666343e-13,4.613820e-13,1.043041e-05,6.678032e-13,2.451106e-13,3.584945e-12,-2.038018e-11,5.397914e-13,1.068943e-06,1.192461e-12,2.279487e-12,3.581202e-12,-7.216152e-13,3.926263e-13,1.983700e-06,1.545089e-13,8.108439e-13,1.719704e-14,-1.355369e-12,4.675319e-13,1.128586e-06,6.616719e-14,9.771079e-13,4.160012e-14,1.114951e-11,2.238777e-12,2.069080e-11,1.206394e-12
1,1409846,845593,1,0.0,0.013514,0.0,0.027217,39,45,28,30,0,1,2072,1.181491,0,1,2,0.185695,0.147442,0.333137,0.027379,0.518833,0.480579,1.742439e-06,1.374692e-06,0.000898,0.000847,6.921463e-13,1.986069e-13,4.286524e-14,-7.284569e-11,3.248140e-11,9.361345e-05,8.865912e-12,9.589357e-11,1.276962e-11,-3.204443e-11,3.572419e-12,2.380382e-05,1.764127e-12,6.267384e-11,3.664161e-12,-2.407480e-11,1.530304e-11,9.363963e-05,3.874162e-12,2.313119e-11,3.278077e-12,-8.453683e-12,2.940329e-12,2.857600e-05,6.475477e-12,2.942376e-10,7.989683e-13,2.228357e-09,2.675846e-09,8.765929e-09,6.802179e-10
2,396322,149376,1,0.0,0.000000,0.0,0.000000,83,40,12,5,0,0,1152,0.000000,1,1,4,0.277350,0.156174,0.433524,0.043315,0.710874,0.589698,7.515437e-06,1.841841e-06,0.001058,0.000778,8.550374e-13,5.111993e-14,4.854499e-14,-8.915369e-11,3.973980e-11,1.422564e-09,3.800342e-09,1.634233e-09,1.577485e-11,-2.793546e-11,1.730548e-12,5.936181e-09,3.561281e-10,4.696963e-11,9.431274e-13,-1.720010e-11,2.884477e-11,1.490348e-07,3.580888e-08,1.094387e-09,4.278142e-11,-8.284127e-12,2.095593e-11,1.365461e-08,9.258149e-10,1.832441e-11,9.048338e-13,9.877339e-18,2.068186e-15,3.498894e-16,8.138709e-17
3,1814022,1791177,1,0.0,0.000000,0.0,0.000000,1,1,2,4,0,0,4,0.000000,1,0,-1,0.577350,0.707107,1.284457,0.408248,1.861807,1.991564,2.848986e-07,8.815667e-07,0.000735,0.000739,0.000000e+00,5.132290e-22,0.000000e+00,-8.684146e-20,2.724252e-20,3.203752e-20,-2.134539e-20,-7.157906e-21,-5.906794e-21,-4.935456e-19,4.270668e-19,1.201956e-15,1.954941e-18,3.802028e-20,6.048757e-20,-4.999468e-21,4.255337e-21,1.166742e-17,1.868446e-20,3.037791e-22,4.297382e-22,-1.532090e-21,7.853008e-22,1.902937e-22,-4.362955e-22,5.882412e-22,-8.762249e-22,3.851983e-35,2.222888e-39,3.739417e-37,2.289326e-37
4,1825190,1686004,1,0.0,0.000000,0.0,0.000000,2,3,1,1,0,0,3,0.000000,1,0,-1,0.707107,0.500000,1.207107,0.353553,1.914214,1.707107,8.815667e-07,3.246765e-07,0.000739,0.000735,3.675139e-18,0.000000e+00,2.584445e-20,-3.482731e-14,7.454880e-15,9.842030e-13,2.495123e-14,5.135090e-14,6.759265e-17,-6.964461e-20,7.177758e-20,-4.418104e-20,-3.040501e-20,1.573541e-19,-2.142644e-19,-3.018256e-21,1.689432e-21,-6.239815e-24,-9.394175e-22,2.016868e-21,-2.601438e-21,-3.527901e-16,7.428118e-17,9.553685e-15,2.384728e-16,4.102897e-16,4.802167e-19,-3.321534e-32,1.732916e-36,1.915235e-34,-3.349830e-34


**Note:** 302 rows in test and CV dataset are common. 

In [35]:
if os.path.isfile(d+'final_test.csv'):
    df_test = pd.read_csv(d+'final_test.csv')
    df_cv   = pd.read_csv(d+'final_cv.csv')
    print('Shape of train data is: ', df_test.shape)
    print('Shape of CV data is: ', df_cv.shape)
    
else:
    # selecting only radom rows: #https://stackoverflow.com/a/22259008/4084039
    n_test = 3775008
    s1 = 50000 #desired sample size: i.e. 50k
    s2 = 25000 #size of cv dataset
    skip_test = sorted(random.sample(range(1,n_test+1),n_test-s1))
    skip_cv   = sorted(random.sample(range(1,n_test+1),n_test-s2))
    
    print("Number of rows in the test data file:", n_test)
    print("Number of rows we are going to elimiate in test data are:",len(skip_test))
        
    print('Numeber of common rows in CV and test data: ', len(set(np.arange(n_test+1))-set(set(skip_cv)|set(skip_test))))
    df_test = pd.read_csv(d+'after_eda/test_after_eda.csv', skiprows=skip_test, names=['source_node', 'destination_node'])
    df_test['indicator_link'] = pd.read_csv(d+'after_eda/test_y.csv', skiprows=skip_test, names=['indicator_link'])
    
    df_cv = pd.read_csv(d+'after_eda/test_after_eda.csv', skiprows=skip_cv, names=['source_node', 'destination_node'])
    df_cv['indicator_link'] = pd.read_csv(d+'after_eda/test_y.csv', skiprows=skip_cv, names=['indicator_link'])
    
    print("Shape of data before featurization ",df_test.shape)
    
    print('Featurizing CV data')
    df_cv    = featurize_my_df(df_cv)
    df_cv.to_csv(d+'final_cv.csv', index = False) 
    
    print('Featurizing test data')
    df_test  = featurize_my_df(df_test)
    df_test.to_csv(d+'final_test.csv', index = False) 
    
    print('Shape of CV data after featurization is  : ', df_cv.shape)
    print('Shape of test data after featurization is: ', df_test.shape)


Shape of train data is:  (50001, 59)
Shape of CV data is:  (25001, 59)


In [36]:
df_test.head()

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,preferntial_attachment,adar_index,follows_back,same_comp,shortest_path,weight_in,weight_out,weight_f1,weight_f2,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,svd_u_s_1,svd_u_s_2,svd_u_s_3,svd_u_s_4,svd_u_s_5,svd_u_s_6,svd_u_d_1,svd_u_d_2,svd_u_d_3,svd_u_d_4,svd_u_d_5,svd_u_d_6,svd_v_s_1,svd_v_s_2,svd_v_s_3,svd_v_s_4,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,svd_u_s_d,svd_v_s_d,svd_u_v_s,svd_u_v_d
0,848424,784690,1,0.052632,0.0,0.109109,0.0,6,6,14,9,1,0,133,0.872503,1,1,2,0.258199,0.377964,0.636163,0.097590,0.894362,1.014128,6.557971e-07,1.559547e-06,0.000754,0.000786,3.243237e-16,1.745627e-16,9.269213e-14,-9.987980e-12,2.283698e-13,1.439960e-10,6.136176e-13,4.188171e-13,5.983441e-15,-1.026186e-11,5.316647e-13,5.843192e-10,1.166050e-13,2.253356e-11,3.220456e-15,-2.148852e-13,1.883259e-13,5.904813e-11,2.701538e-12,4.341620e-13,5.535503e-14,-9.994077e-10,5.791914e-10,3.512351e-07,2.486660e-09,2.771146e-09,1.727695e-12,8.425175e-20,2.074802e-17,8.506723e-21,2.053067e-16
1,1160264,1270588,1,0.555556,0.0,0.745356,0.0,9,25,5,0,5,0,150,3.279655,0,1,2,0.408248,0.196116,0.604364,0.080064,1.012613,0.800481,4.419306e-07,2.418359e-07,0.000765,0.000750,2.433689e-195,0.000000e+00,1.263933e-194,5.970698e-19,-2.040362e-19,4.739243e-19,-6.512985e-19,-2.917969e-20,5.854696e-19,-1.543385e-19,3.409813e-20,3.134960e-20,8.127028e-20,-3.397369e-20,-8.850713e-20,2.050807e-20,-1.814967e-20,3.209467e-20,-4.610276e-20,4.480243e-21,2.885330e-20,8.381996e-21,-1.391114e-20,2.260277e-20,-3.184752e-20,3.210733e-21,1.935941e-20,-1.880089e-37,3.191036e-39,7.794704e-38,-5.470204e-39
2,766166,850674,1,0.000000,0.0,0.000000,0.0,3,5,6,4,0,0,42,0.000000,0,1,3,0.377964,0.408248,0.786213,0.154303,1.164177,1.194461,3.680705e-07,7.959495e-07,0.000743,0.000754,1.499315e-15,2.365667e-16,2.348261e-17,-2.168921e-14,3.138449e-13,7.902062e-13,6.651568e-12,4.764876e-12,2.766118e-14,-2.036965e-14,1.717373e-14,8.833559e-12,6.159764e-12,1.168462e-11,4.364386e-15,-2.235688e-15,3.068982e-13,2.306734e-13,2.782631e-12,4.796504e-14,4.431302e-13,-9.841215e-16,9.238453e-15,4.377835e-14,4.561890e-13,6.147218e-14,4.376945e-16,1.036341e-22,1.285484e-24,1.902831e-23,3.915194e-24
3,1730930,1021730,1,0.000000,0.0,0.000000,0.0,8,7,43,42,0,0,513,0.000000,1,1,7,0.150756,0.353553,0.504309,0.053300,0.655065,0.857862,8.682250e-07,4.978000e-06,0.000762,0.000898,5.731288e-18,4.151718e-13,2.304104e-13,-1.191717e-15,6.296930e-14,1.473531e-13,7.013950e-14,3.998318e-15,1.056584e-16,-2.985524e-11,2.012668e-10,1.553877e-09,2.026831e-10,3.941107e-10,7.659634e-12,-3.034200e-13,2.441602e-12,1.515123e-11,2.438649e-12,3.152527e-12,5.580904e-14,-8.674741e-12,3.357381e-10,1.110601e-09,2.150155e-11,1.482890e-09,4.294637e-12,2.574705e-22,2.237688e-20,2.570345e-24,2.382383e-18
4,1454639,713591,1,0.000000,0.0,0.000000,0.0,0,0,1,1,0,0,0,0.000000,0,0,-1,0.707107,0.601457,1.308564,0.425294,2.015671,1.910021,5.615700e-07,2.848986e-07,0.000748,0.000735,0.000000e+00,2.554745e-17,8.520731e-18,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.791217e-14,2.955285e-15,1.356772e-08,9.265312e-16,5.217553e-15,4.713970e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.189441e-13,7.258514e-15,4.536167e-08,1.763661e-15,5.147947e-15,1.588197e-16,0.000000e+00,0.000000e+00,0.000000e+00,6.154546e-16


In [37]:
df_cv.head()

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,preferntial_attachment,adar_index,follows_back,same_comp,shortest_path,weight_in,weight_out,weight_f1,weight_f2,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,svd_u_s_1,svd_u_s_2,svd_u_s_3,svd_u_s_4,svd_u_s_5,svd_u_s_6,svd_u_d_1,svd_u_d_2,svd_u_d_3,svd_u_d_4,svd_u_d_5,svd_u_d_6,svd_v_s_1,svd_v_s_2,svd_v_s_3,svd_v_s_4,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,svd_u_s_d,svd_v_s_d,svd_u_v_s,svd_u_v_d
0,848424,784690,1,0.052632,0.000000,0.109109,0.000000,6,6,14,9,1,0,133,0.872503,1,1,2,0.258199,0.377964,0.636163,0.097590,0.894362,1.014128,6.557971e-07,1.559547e-06,0.000754,0.000786,3.243237e-16,1.745627e-16,9.269213e-14,-9.987980e-12,2.283698e-13,1.439960e-10,6.136176e-13,4.188171e-13,5.983441e-15,-1.026186e-11,5.316647e-13,5.843192e-10,1.166050e-13,2.253356e-11,3.220456e-15,-2.148852e-13,1.883259e-13,5.904813e-11,2.701538e-12,4.341620e-13,5.535503e-14,-9.994077e-10,5.791914e-10,3.512351e-07,2.486660e-09,2.771146e-09,1.727695e-12,8.425175e-20,2.074802e-17,8.506723e-21,2.053067e-16
1,1494573,1088031,1,0.000000,0.000000,0.000000,0.000000,2,2,2,5,0,0,20,1.430677,1,1,6,0.577350,0.577350,1.154701,0.333333,1.732051,1.732051,3.803655e-07,2.037218e-07,0.000739,0.000739,1.352607e-17,9.758370e-17,2.043969e-15,-1.268575e-14,5.081077e-15,6.128186e-09,2.637256e-14,6.800240e-15,2.495802e-16,-3.888018e-14,1.625755e-14,1.651897e-07,6.761970e-15,1.829764e-13,1.800353e-15,-1.670641e-18,1.180784e-15,7.308777e-08,7.858512e-16,2.211163e-15,5.638269e-17,-1.201278e-14,1.220945e-14,1.674028e-07,2.486195e-14,3.649407e-14,3.809769e-14,1.012313e-15,1.223510e-14,4.478954e-16,2.765323e-14
2,1480280,1221992,1,0.025641,0.037037,0.160128,0.138675,39,26,1,2,1,1,98,0.706727,1,1,2,0.707107,0.192450,0.899557,0.136083,1.606664,1.092007,2.428049e-06,1.872620e-07,0.000887,0.000735,5.149650e-15,5.242421e-16,2.778235e-17,-6.981071e-13,4.471772e-08,2.261156e-07,1.574584e-12,4.752161e-12,9.500738e-14,-1.024079e-13,4.796267e-11,1.464332e-09,6.492712e-13,9.746253e-14,9.671808e-15,-7.926587e-12,3.628682e-09,1.484885e-07,5.596091e-11,1.110414e-11,1.342510e-12,-6.788610e-15,3.502357e-12,2.237453e-11,1.040298e-12,2.694240e-14,5.178373e-16,3.332529e-16,3.335127e-18,3.373782e-14,3.293239e-20
3,199064,100138,1,0.000000,0.000000,0.000000,0.000000,5,5,0,3,0,0,21,0.000000,0,1,-1,1.000000,0.408248,1.408248,0.408248,2.408248,1.816497,4.956266e-07,1.655650e-07,0.000750,0.000731,2.819259e-19,8.539501e-18,0.000000e+00,-4.197272e-15,2.311236e-12,1.660947e-13,7.988226e-16,1.175672e-15,5.299641e-18,-2.873520e-14,4.404743e-13,1.264190e-12,3.511241e-15,9.217326e-14,1.575650e-16,-6.013204e-16,5.291141e-13,2.321092e-14,2.726608e-15,6.568976e-15,1.809385e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.228247e-24,0.000000e+00,1.226775e-24,0.000000e+00
4,796220,1700343,1,0.025000,0.014925,0.058926,0.048393,9,7,32,61,1,1,715,1.300501,1,1,3,0.174078,0.353553,0.527631,0.061546,0.701709,0.881184,3.383960e-07,1.382374e-06,0.000769,0.000865,5.035257e-13,8.937457e-12,4.210956e-11,-8.241874e-11,1.448558e-11,4.492112e-10,2.715749e-11,2.970341e-11,9.289701e-12,-4.346418e-10,6.175454e-11,7.641265e-08,6.993390e-10,4.124855e-08,1.648899e-10,-4.251895e-11,6.393418e-11,1.888170e-09,1.220502e-10,9.714286e-10,2.512368e-10,-1.085707e-09,1.210319e-10,8.962166e-09,2.662319e-10,1.080588e-09,7.848833e-10,3.560788e-17,1.825540e-17,8.871210e-19,7.301905e-16
